# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import subprocess
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.core.display_functions import update_display
from IPython.display import Markdown, display

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [4]:
#Model detection function
def get_ollama_models():
    """Detect available local models"""
    try:
        ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
        models = ollama_client.models.list()
        return [models.id for model in models.data]
    except:
        try:
            result = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
            lines = result.stdout.strip().split('\n')[1:]  
            models = []
            for line in lines:
                if line.strip():
                    model_name = line.split()[0]
                    models.append(model_name)
            return models
        except:
            return []

In [ ]:
def check_ollama_running():
    """Check if Ollama service is running"""
    try:
        import requests
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        return response.status_code == 200
    except:
        return False

available_models = []

# Always add OpenAI models if API key is valid
if api_key and api_key.startswith("sk-proj-"):
    available_models.append(("OpenAI", MODEL_GPT))

# Check for Ollama and its models
ollama_running = check_ollama_running()
if ollama_running:
    ollama_models = get_ollama_models()
    for model in ollama_models:
        available_models.append(("Ollama", model))
    print(f" Found {len(ollama_models)} Ollama model(s): {', '.join(ollama_models)}")
else:
    print(" Ollama is not running. Start it with 'ollama serve' to use local models")

print(f"\n Total available models: {len(available_models)}")
for i, (provider, model) in enumerate(available_models, 1):
    print(f"   {i}. [{provider}] {model}")


In [ ]:
selected_index = 2  # Change this to select different model (1-based index)

if 1 <= selected_index <= len(available_models):
    provider, selected_model = available_models[selected_index - 1]
    print(f"Selected: [{provider}] {selected_model}")
else:
    provider, selected_model = "OpenAI", MODEL_GPT
    print(f"  Invalid selection, defaulting to: [{provider}] {selected_model}")

# Question input - edit this cell to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

# Optional: You can leave empty to get a default behavior
if not question.strip():
    question = "Explain the concept of decorators in Python with examples"

messages = [{"role": "user", "content": question}]
print(f"\n Question: {question[:100]}..." if len(question) > 100 else f"\n Question: {question}")

In [ ]:
def stream_response(provider, model, messages):
    """Stream response from either OpenAI or Ollama"""
    
    # Initialize appropriate client
    if provider == "OpenAI":
        client = OpenAI()  # Uses API key from environment
    else:  # Ollama
        client = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
    
    try:
        # Create streaming completion
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True,
            temperature=0.7  # You can adjust this
        )
        
        # Stream the response
        response = ""
        display_handle = display(Markdown("▌"), display_id=True)
        
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                # Update display with streaming indicator
                update_display(Markdown(response + " ▌"), 
                             display_id=display_handle.display_id)
        
        # Final update without the streaming indicator
        update_display(Markdown(response), 
                      display_id=display_handle.display_id)
        
        return response
        
    except Exception as e:
        error_msg = f" Error with {provider} model '{model}': {str(e)}"
        print(error_msg)
        return None

# Stream the response
print(f"\n Getting response from [{provider}] {selected_model}...\n")
response = stream_response(provider, selected_model, messages)